In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from pathlib import Path
import os
from dotenv import load_dotenv

# 1. 환경변수 로드 (.env 파일에 OPENAI_API_KEY=your_api_key 형태로 저장)
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("❌ OpenAI API 키가 환경변수에 없습니다. .env 파일을 확인하세요.")

# 2. PDF 절대 경로 지정
pdf_path = Path(r"C:\woohyun\AJIN-12th-project\data\컴활1급.pdf")

# 3. PDF 파일 존재 여부 체크
if not pdf_path.exists():
    raise FileNotFoundError(f"PDF 파일이 없습니다: {pdf_path}")

# 4. PDF 문서 로드
loader = PyPDFLoader(str(pdf_path))
all_docs = loader.load()

# 5. PDF 문서를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
split_docs = text_splitter.split_documents(all_docs)

# 6. 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 7. 벡터스토어 생성 (FAISS 인덱스 생성)
vectorstore = FAISS.from_documents(split_docs, embeddings)

# 8. 벡터스토어 저장 폴더 생성
save_dir = Path.cwd() / "faiss_db"
save_dir.mkdir(parents=True, exist_ok=True)

# 9. 벡터스토어 로컬 저장
vectorstore.save_local(folder_path=str(save_dir), index_name="index")
print(f"✅ 벡터스토어 저장 완료: {save_dir.resolve()}")

# 10. 저장된 벡터스토어 불러오기 (보안 옵션 True 설정)
vectorstore = FAISS.load_local(
    str(save_dir),
    embeddings,
    index_name="index",
    allow_dangerous_deserialization=True
)

# 11. 검색기 생성 (유사도 검색, k=3)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 12. 프롬프트 템플릿 생성
prompt_template = """
당신은 친절하고 유능한 전문가입니다. 다음은 사용자의 질문과 관련된 참고 문서입니다:

{context}

질문: {question}

참고 문서를 기반으로 질문에 정확하게 답하세요. 모르는 내용은 추측하지 마세요.
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# 13. 언어 모델 초기화
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=openai_api_key)

# 14. RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

# 15. 테스트 질의
query = "엑셀에서 함수 IF의 기본적인 사용법은 무엇인가요?"
response = qa_chain.invoke({"query": query})

print("🔍 질문:", query)
print("💬 답변:", response)


✅ 벡터스토어 저장 완료: C:\woohyun\AJIN-12th-project\faiss_db
🔍 질문: 엑셀에서 함수 IF의 기본적인 사용법은 무엇인가요?
💬 답변: {'query': '엑셀에서 함수 IF의 기본적인 사용법은 무엇인가요?', 'result': '엑셀에서 함수 `IF`의 기본적인 사용법은 특정 조건을 평가하여 그 조건이 참일 경우와 거짓일 경우에 따라 다른 결과를 반환하는 것입니다. `IF` 함수의 기본 구문은 다음과 같습니다:\n\n```\nIF(조건, 참일_경우_반환값, 거짓일_경우_반환값)\n```\n\n- `조건`: 평가할 논리적 조건입니다. 이 조건이 참인지 거짓인지에 따라 반환값이 결정됩니다.\n- `참일_경우_반환값`: 조건이 참일 때 반환할 값입니다.\n- `거짓일_경우_반환값`: 조건이 거짓일 때 반환할 값입니다.\n\n예를 들어, 셀 A1의 값이 10보다 크면 "크다"를, 그렇지 않으면 "작다"를 반환하는 수식은 다음과 같습니다:\n\n```\nIF(A1 > 10, "크다", "작다")\n```\n\n이 수식은 A1의 값이 10보다 큰 경우 "크다"를 반환하고, 그렇지 않으면 "작다"를 반환합니다.'}
